In [4]:
# ==========================================
# CELL 1: Setup & Data Download (K Scott Mader Dataset)
# ==========================================
import os
from google.colab import files

# 1. Install necessary libraries
!pip install ultralytics -q
!pip install pandas -q

# 2. Upload kaggle.json
print("Please upload your kaggle.json file:")
uploaded = files.upload()

# 3. Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/.kaggle.json # Fix: Ensure kaggle.json is copied as .kaggle.json
!chmod 600 ~/.kaggle/.kaggle.json

# 4. Download K Scott Mader Malaria Dataset
print("Downloading K Scott Mader Malaria Dataset...")
!kaggle datasets download -d kmader/malaria-bounding-boxes
!mkdir -p datasets/raw_malaria  # Create the directory before unzipping
!unzip -q malaria-bounding-boxes.zip -d datasets/raw_malaria

print("Download Complete.")

Please upload your kaggle.json file:


Saving kaggle.json to kaggle (2).json
chmod: cannot access '/root/.kaggle/.kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/kmader/malaria-bounding-boxes
License(s): Attribution-NonCommercial-ShareAlike 3.0 IGO (CC BY-NC-SA 3.0 IGO)
malaria-bounding-boxes.zip: Skipping, found more recently modified local copy (use --force to force download)
Download Complete.


In [5]:
# ==========================================
# CELL 2: Convert JSON to YOLO Format
# ==========================================
import json
import pandas as pd
import os
import shutil
import cv2
from tqdm.notebook import tqdm

# Define paths
base_path = 'datasets/raw_malaria/malaria'
images_source = os.path.join(base_path, 'images')
json_train = os.path.join(base_path, 'training.json')
json_test = os.path.join(base_path, 'test.json')

# Output directories for YOLO
output_path = 'datasets/malaria_yolo'
for split in ['train', 'test']:
    os.makedirs(f'{output_path}/images/{split}', exist_ok=True)
    os.makedirs(f'{output_path}/labels/{split}', exist_ok=True)

# Define Classes (Based on dataset analysis)
class_map = {
    'red blood cell': 0,
    'trophozoite': 1,
    'difficult': 2,
    'ring': 3,
    'schizont': 4,
    'gametocyte': 5,
    'leukocyte': 6
}

def process_json_to_yolo(json_file, split_name):
    print(f"Processing {split_name} data...")
    df = pd.read_json(json_file)

    # Iterate through each image entry
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        # Get image info
        img_data = row['image']
        filename = os.path.basename(img_data['pathname'])

        # Source image path (handle potential subdirectories if any, usually flat in this dataset)
        src_img_path = os.path.join(images_source, filename)

        # Check if image exists
        if not os.path.exists(src_img_path):
            continue

        # Read image to get dimensions (needed for normalization)
        img = cv2.imread(src_img_path)
        if img is None: continue
        height, width, _ = img.shape

        # Copy image to YOLO folder
        dst_img_path = f'{output_path}/images/{split_name}/{filename}'
        shutil.copy(src_img_path, dst_img_path)

        # Create Label File
        label_filename = filename.rsplit('.', 1)[0] + '.txt'
        label_path = f'{output_path}/labels/{split_name}/{label_filename}'

        with open(label_path, 'w') as f:
            for obj in row['objects']:
                category = obj['category']
                if category not in class_map: continue
                cls_id = class_map[category]

                # Extract Box (JSON format: r=row(y), c=col(x))
                bbox = obj['bounding_box']
                min_r, min_c = bbox['minimum']['r'], bbox['minimum']['c']
                max_r, max_c = bbox['maximum']['r'], bbox['maximum']['c']

                # Convert to YOLO (x_center, y_center, w, h) normalized
                x_center = ((min_c + max_c) / 2) / width
                y_center = ((min_r + max_r) / 2) / height
                w = (max_c - min_c) / width
                h = (max_r - min_r) / height

                # Clip values to 0-1 just in case
                x_center = max(0, min(1, x_center))
                y_center = max(0, min(1, y_center))
                w = max(0, min(1, w))
                h = max(0, min(1, h))

                f.write(f"{cls_id} {x_center} {y_center} {w} {h}\n")

# Run conversion
process_json_to_yolo(json_train, 'train')
process_json_to_yolo(json_test, 'test')

# Create data.yaml
data_yaml_content = f"""
path: /content/{output_path}
train: images/train
val: images/test  # Using test as val for simplicity here
test: images/test
nc: {len(class_map)}
names: {list(class_map.keys())}
"""
with open(f'{output_path}/data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Dataset conversion to YOLO format complete!")

Processing train data...


  0%|          | 0/1208 [00:00<?, ?it/s]

Processing test data...


  0%|          | 0/120 [00:00<?, ?it/s]

Dataset conversion to YOLO format complete!


In [6]:
# ==========================================
# CELL 3: Define CBAM Attention (Like Platelet.ipynb)
# ==========================================
import torch
import torch.nn as nn
import ultralytics.nn.tasks

# 1. Channel Attention Module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

# 2. Spatial Attention Module
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

# 3. CBAM Block
class CBAM(nn.Module):
    def __init__(self, c1, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        return x * self.channel_attention(x) * self.spatial_attention(x)

# 4. Register CBAM
# This allows the YAML file to find "CBAM"
ultralytics.nn.tasks.CBAM = CBAM
# Sometimes required for safe lookup
setattr(ultralytics.nn.modules, 'CBAM', CBAM)

print("Attention Mechanism Registered Successfully.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Attention Mechanism Registered Successfully.


In [9]:
# ==========================================
# CELL 4: Custom Model Config with Attention
# ==========================================
# We insert the CBAM module into the YOLOv8n backbone
# NOTE: The [1024] argument for CBAM matches the channels of the layer it follows

yolo_cbam_yaml = """
# Ultralytics YOLOv8n-CBAM model
nc: 7
scales:
  n: [0.33, 0.25, 1024]

backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]  # 9
  - [-1, 1, CBAM, [256]]     # 10 <-- ATTENTION ADDED HERE, changed from 1024 to 256

head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [512]]  # 13

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [256]]  # 16 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 13], 1, Concat, [1]]  # cat head P4
  - [-1, 3, C2f, [512]]  # 19 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]]  # cat head P5 (Connects to CBAM layer)
  - [-1, 3, C2f, [1024]]  # 22 (P5/32-large)

  - [[16, 19, 22], 1, Detect, [nc]]  # Detect(P3, P4, P5)
"""

with open("yolov8_cbam.yaml", "w") as f:
    f.write(yolo_cbam_yaml)

print("yolov8_cbam.yaml created.")

yolov8_cbam.yaml created.


In [10]:
# ==========================================
# CELL 5: Train
# ==========================================
from ultralytics import YOLO

# Load the custom config
model = YOLO('yolov8_cbam.yaml')

# Train
results = model.train(
    data='datasets/malaria_yolo/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='malaria_attention_run',
    patience=10
)

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.246 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/malaria_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8_cbam.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=malaria_attention_run, nbs=64, nms=False, opset=None, optimize=False, optim

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/50      6.74G      5.081      3.285      3.871        668        640: 100% ━━━━━━━━━━━━ 76/76 1.1s/it 1:20
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.2s/it 4.6s
                   all        120       5922          0          0          0          0

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/50      6.74G      3.624      1.921      3.163       1960        640: 0% ──────────── 0/76  0.5s

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/50      6.76G      2.353      1.308      2.295        922        640: 100% ━━━━━━━━━━━━ 76/76 1.1it/s 1:12
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.2it/s 1.8s
                   all        120       5922      0.937     0.0413     0.0532     0.0264

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/50      6.76G      1.726      1.076      1.825       1585        640: 0% ──────────── 0/76  0.8s

...

      48/50      5.95G     0.7764     0.4438      0.954        480        640: 100% ━━━━━━━━━━━━ 76/76 1.1it/s 1:11
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.4it/s 1.6s
                   all        120       5922      0.405      0.435      0.389      0.312

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      49/50      5.96G     0.7785     0.4388     0.9869       1002        640: 0% ──────────── 0/76  1.0s

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      49/50      5.96G     0.7709     0.4398     0.9582        554        640: 100% ━━━━━━━━━━━━ 76/76 1.1it/s 1:12
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.3it/s 1.7s
                   all        120       5922      0.369      0.386      0.389      0.311

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      50/50      5.97G      0.824     0.4721     0.9617       1069        640: 0% ──────────── 0/76  0.5s

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:841: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:157.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      50/50      5.98G     0.7728     0.4401     0.9585        429        640: 100% ━━━━━━━━━━━━ 76/76 1.1it/s 1:11
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.4it/s 1.7s
                   all        120       5922      0.364      0.387      0.386      0.308

50 epochs completed in 1.046 hours.
Optimizer stripped from /content/runs/detect/malaria_attention_run/weights/last.pt, 6.3MB
Optimizer stripped from /content/runs/detect/malaria_attention_run/weights/best.pt, 6.3MB

Validating /content/runs/detect/malaria_attention_run/weights/best.pt...
Ultralytics 8.3.246 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8_cbam summary (fused): 80 layers, 3,015,303 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.7s/it 6.7s
                   all        120       5922      0.402        0.4      0.394      0.

In [13]:
# ==========================================
# CELL 6: Platelet-Style "True vs Pred" Counting
# ==========================================
import glob
import os

# Paths
test_images = 'datasets/malaria_yolo/images/test'
test_labels = 'datasets/malaria_yolo/labels/test'

# Load best model
best_model = YOLO('runs/detect/malaria_attention_run/weights/best.pt')

print("\n" + "="*95)
print(f"{'Image Name':<40} | {'True Objects':<12} | {'Pred Objects':<12} | {'Diff':<5}")
print("="*95)

total_true = 0
total_pred = 0

# Just check the first 20 images to avoid spamming output, remove slice [:20] for full
image_files = glob.glob(os.path.join(test_images, '*.*'))

for img_file in image_files[:30]:
    filename = os.path.basename(img_file)
    label_file = os.path.join(test_labels, filename.rsplit('.', 1)[0] + '.txt')

    # 1. Get True Count (excluding RBCs usually, or counting all)
    # Note: Class 0 is RBC. In Malaria detection, we usually care about Parasites (Classes 1,3,4,5)
    # Let's count TOTAL objects for general detection, or modify 'if' below to filter
    true_count = 0
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            lines = f.readlines()
            # To count ONLY parasites (exclude RBC/Leukocyte), uncomment next line:
            # lines = [l for l in lines if l.split()[0] not in ['0', '6']]
            true_count = len(lines)

    # 2. Predict
    # conf=0.25 is standard confidence threshold
    results = best_model.predict(img_file, verbose=False, conf=0.25)

    # Filter predictions same as ground truth if needed
    # boxes.cls gives class IDs.
    pred_boxes = results[0].boxes
    # To count ONLY parasites:
    # pred_count = len([c for c in pred_boxes.cls if int(c) not in [0, 6]])
    pred_count = len(pred_boxes)

    # 3. Stats
    diff = pred_count - true_count
    total_true += true_count
    total_pred += pred_count

    print(f"{filename:<40} | {true_count:<12} | {pred_count:<12} | {diff:<5}")

print("="*95)
print(f"Total (Sample): True={total_true} | Predicted={total_pred}")
if total_true > 0:
    print(f"Error Rate: {abs(total_true - total_pred) / total_true:.2%}")

# ==========================================
# CELL 7: Download Runs Folder
# ==========================================
import shutil
from google.colab import files

# Define the path to the 'runs' folder
runs_folder_path = 'runs'
output_zip_name = 'runs_folder.zip'

# Create a zip archive of the 'runs' folder
print(f"Compressing '{runs_folder_path}' into '{output_zip_name}'...")
# Correcting the base_name for make_archive to ensure the output file has the desired name
shutil.make_archive(output_zip_name.rsplit('.', 1)[0], 'zip', runs_folder_path)

# Provide the download link
print(f"'{output_zip_name}' created. Initiating download...")
files.download(output_zip_name)

print("Download initiated. Please check your browser's download location.")


Image Name                               | True Objects | Pred Objects | Diff 
bd4da4f8-8996-47e0-a4e5-a3da5717441e.jpg | 13           | 17           | 4    
010961af-b38c-49de-aca0-e3732d73d414.jpg | 58           | 64           | 6    
9cf0b006-cb5c-47e7-b076-1dcacf1fbfb1.jpg | 50           | 59           | 9    
f0ee03d6-c57b-43a0-8812-359330bdb93a.jpg | 104          | 47           | -57  
5a2447bf-a92b-4a47-a384-4b2ff020ecdc.jpg | 39           | 36           | -3   
03be08c8-2cf3-4de6-8ac6-5f7817a6a643.jpg | 55           | 61           | 6    
baab0701-3fd4-4940-8b90-b9c42d95888f.jpg | 70           | 70           | 0    
e26b88c6-5884-4b5c-94d7-ad780379c8ad.jpg | 70           | 78           | 8    
b1d312cf-1f52-4955-bf42-58d3664c254a.jpg | 50           | 56           | 6    
c8d1b7ec-be9f-4c88-a39b-2c6bf46b2998.jpg | 47           | 52           | 5    
efb85f8e-c780-4a19-b6ce-555a154b0d10.jpg | 84           | 77           | -7   
8874ea02-d263-4830-99d1-e1b14230a56b.jpg | 49      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated. Please check your browser's download location.
